In [38]:
import pandas as pd

filename = '../big_a_data/all.csv'
df = pd.read_csv(filename, converters={'code': str})
#df = df[df['code'].isin(['000538', '601601', '601928', '600750', '601728', '601166', '600016', '600282', '601318', '600737', '000895', '001965', '600941', '601127', '600062', '600660', '600036', '600887', '601107', '000999', '002179', '000063', '000858'])]
df = df[~df['code'].str.startswith(('300', '688'))] #去掉创业板和科板
DS_NAME = 'all'
#DS_NAME = '自选'
print('Done', len(df))
df = df[['code', 'name', '股息率_2024']].copy()
#df.head(3)

Done 3810


In [39]:
str_today = '2025-05-23'
DURATION = 330
DAYS_PRE = 2
DAYS_NEXT = 5
ZHANG = 0.5
cols_pre = [f'd-{i}' for i in range(DAYS_PRE, 0, -1)]
cols_next = [f'd{i}' for i in range(1, DAYS_NEXT + 1)]
filename_output = f'涨买_前{DAYS_PRE}+日跌_今涨_收盘价买入_后{DAYS_NEXT}日_涨{ZHANG}_{str_today}_{DS_NAME}.csv'
df_out = pd.DataFrame(columns=['code', 'select', 'get', 'rate', 'profit_min'])

for code in df['code']:
    print(code, end='\r')
    df_process = pd.read_csv(f'../big_a_ds/process_daily_2nd/{code}.csv')
    if df_process.iloc[-1]['日期'] != str_today: continue #停牌票
    if len(df_process) < DURATION + DAYS_PRE + DAYS_NEXT + DAYS_NEXT: continue #不够数量
    
    #prepare data
    df_select = df_process[-DURATION - DAYS_NEXT : -DAYS_NEXT].copy()
    df_select[f'后{DAYS_NEXT}日最大涨幅'] = df_select[cols_next].max(axis=1)
    
    #buy condition
    df_select = df_select[df_select['收盘'] > df_select['开盘']]
    df_select = df_select[df_select[cols_pre].eq('green').all(axis=1)]
    
    #get
    df_get = df_select[df_select[f'后{DAYS_NEXT}日最大涨幅'] >= ZHANG]
    
    #record
    if len(df_get) > 0:
        new_row = {'code': code, 
                   'select': len(df_select), 
                   'get': len(df_get), 
                   'rate': len(df_get) / len(df_select) * 100, 
                   'profit_min': df_get[f'后{DAYS_NEXT}日最大涨幅'].min()}
        df_out.loc[len(df_out)] = new_row

df_out = pd.merge(df, df_out, on='code', how='inner')
print('\r\nDone', len(df_out))
#print(df_out)
df_out['rate'] = df_out['rate'].round(2)
df_out.sort_values(by=['rate', 'select'], ascending=False, inplace=True)
df_out.to_csv(filename_output, index=False)

873833
Done 3736


df_temp = pd.read_csv(filename_output, converters={'code': str})
#df_temp = df_temp[['code', 'name', '股息率_2024', 'select', 'get', 'rate', 'profit_min']].copy()
df_temp = df_temp[df_temp['股息率_2024'] > 3]
#df_temp = df_temp[df_temp['select'] > 30]
df_temp = df_temp[df_temp['rate'] > 90]
df_temp.sort_values(by=['rate', 'select'], ascending=False, inplace=True)
print(df_temp)
print(len(df_temp))
df_temp.to_csv('关注_' + filename_output, index=False)